<a href="https://colab.research.google.com/github/ShivamThapa243/Information-Retrieval/blob/main/positional_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Positional Inverted Index

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Check Point: Checking if the files are readable

In [ ]:
path = "/content/drive/MyDrive/Dataset/text_files/file1.txt"

with open(path, 'r') as file:
  content = file.read()

print(content)

Loving these vintage springs on my vintage strat. They have a good tension and great stability. If you are floating your bridge and want the most out of your springs than these are the way to go.


In [ ]:
def summation(num1, num2):
  return num1 + num2

def multiplication(num1, num2):
  return num1 * num2

In [ ]:
print(summation(5,6))
print(multiplication(13,15))

11
195
